In [ ]:
import pandas as pd

import visions as v
from visions.typesets import CompleteSet
from visions.functional import type_inference_frame, type_cast_frame
%matplotlib inline

In [ ]:
# Load dataset
df = pd.read_csv(
    "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
)


df.head()

In [ ]:
typeset = CompleteSet()
typeset.plot_graph()

In [ ]:
inferred_types = type_inference_frame(df, typeset)
inferred_types

In [ ]:
cast_df = type_cast_frame(df, typeset)
cast_df.head()